In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
%cd /content
!rm -rf Real-Time-Anomaly-Segmentation-for-Road-Scenes
!git clone https://github.com/luca-bergamini/Real-Time-Anomaly-Segmentation-for-Road-Scenes.git
%cd Real-Time-Anomaly-Segmentation-for-Road-Scenes
!git checkout Luca-R

/content
Cloning into 'Real-Time-Anomaly-Segmentation-for-Road-Scenes'...
remote: Enumerating objects: 983, done.
remote: Counting objects: 100% (179/179), done.
remote: Compressing objects: 100% (105/105), done.
remote: Total 983 (delta 130), reused 118 (delta 74), pack-reused 804 (from 1)
Receiving objects: 100% (983/983), 386.41 MiB | 42.44 MiB/s, done.
Resolving deltas: 100% (349/349), done.
/content/Real-Time-Anomaly-Segmentation-for-Road-Scenes
Branch 'Luca-R' set up to track remote branch 'Luca-R' from 'origin'.
Switched to a new branch 'Luca-R'


In [8]:
!pip3 install visdom
!pip3 install ood_metrics
!pip3 install torchvision
!pip install fvcore

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fvcore: filename=fvcore-0.1.5.post20221221-py3-none-any.whl size=61397 sha256=25be5861733b58c7a6238c5b074ffbe13660845ca226b6c5acd34305126b02e6
  Stored in directory: /root/.cache/pip/wheels/65/71/95/3b8fde5c65c6e4a806e0867c1651dcc71a1cb2f3430e8f355f
  Created wheel for iopath: filename=iopath-0.1.10-py3-none-any.whl size=31527 sha256=7c6a71a363d82754d734fd35aebf25e122ebed9adf75ecaef10f554c340f8668
  Stored in directory: /root/.cache/pip/wheels/ba/5e/16/6117f8fe7e9c0c161a795e10d94645ebcf301ccbd01f66d8ec
Successfully built fvcore iopath


# Enet Training

In [ ]:
!cd "./train" && python3 main.py \
--model enet \
--datadir "/content/drive/MyDrive/cityscapes" \
--state "../trained_models/enet_pretrained.pth" \
--savedir "/content/drive/MyDrive/training_results/enet/" \
--num-epochs=40 \
--decoder

========== DECODER TRAINING ===========
/content/drive/MyDrive/cityscapes/leftImg8bit/train
/content/drive/MyDrive/cityscapes/leftImg8bit/val
<class '__main__.CrossEntropyLoss2d'>
----- TRAINING - EPOCH 1 -----
/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:227: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:243: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed fo

# EVAL_IOU

In [ ]:
!python3 ./eval/eval_iou.py \
    --loadDir "/content/drive/MyDrive/training_results/enet/" \
    --loadWeights "model_best.pth" \
    --loadModel "./train/enet.py" \
    --subset val \
    --datadir "/content/drive/MyDrive/cityscapes" \
    --batch-size 1 \
    --num-workers 4 \
    --void

/content/drive/MyDrive/cityscapes/leftImg8bit/val /content/drive/MyDrive/cityscapes/gtFine/val
---------------------------------------
Took  189.5460171699524 seconds
Per-Class IoU:
90.05 Road
62.12 sidewalk
80.76 building
30.46 wall
25.89 fence
36.77 pole
21.91 traffic light
40.60 traffic sign
85.62 vegetation
47.47 terrain
87.94 sky
47.84 person
0.00 rider
83.70 car
11.73 truck
29.37 bus
6.49 train
0.16 motorcycle
47.22 bicycle
60.63 void
MEAN IoU:  44.84 %


# Void Class

In [ ]:
import os

for dataset in ['RoadAnomaly21','RoadObsticle21','fs_static','FS_LostFound_full','RoadAnomaly']:
  for method in ['Void']:

    format_file = os.listdir(f'/content/drive/MyDrive/Validation_Dataset/{dataset}/images')[0].split(".")[1]
    input = f'/content/drive/MyDrive/Validation_Dataset/{dataset}/images/\*.{format_file}'
    print(f"\nDataset: {dataset} - method : {method}")

    !python3 ./eval/evalAnomaly.py \
    --input {input} \
    --loadModel './train/enet.py' \
    --method {method} \
    --loadDir "/content/drive/MyDrive/training_results/enet/" \
    --loadWeights 'model_best.pth'



Dataset: RoadAnomaly21 - method : Void
AUPRC score: 14.18350845440059
FPR@TPR95: 85.49306421679567

Dataset: RoadObsticle21 - method : Void
AUPRC score: 1.4896421847847223
FPR@TPR95: 51.67289908451357

Dataset: fs_static - method : Void
AUPRC score: 5.167231726281392
FPR@TPR95: 43.00049303188575

Dataset: FS_LostFound_full - method : Void
AUPRC score: 3.7129541153773613
FPR@TPR95: 75.10682335405883

Dataset: RoadAnomaly - method : Void
AUPRC score: 8.317108475052331
FPR@TPR95: 79.48221203159046


# Pruning

In [10]:
for pruning in [0.1, 0.2, 0.3, 0.4, 0.5]:
    print(f"\nRunning evaluation with pruning={pruning}")
    !python3 ./eval/eval_iou.py \
        --loadDir "/content/drive/MyDrive/training_results/enet/" \
        --loadWeights "model_best.pth" \
        --subset val \
        --datadir "/content/drive/MyDrive/cityscapes" \
        --batch-size 1 \
        --num-workers 4 \
        --void \
        --pruning {pruning}


Running evaluation with pruning=0.1
Applying unstructured L1 pruning with 10.0% sparsity to Conv2d layers...
Total parameters: 351804 | Non-zero (effective) parameters after pruning: 318284
Pruned percentage: 9.53%
Unsupported operator aten::max_pool2d encountered 1 time(s)
Unsupported operator aten::add_ encountered 88 time(s)
Unsupported operator aten::prelu encountered 93 time(s)
Unsupported operator aten::max_pool2d_with_indices encountered 2 time(s)
Unsupported operator aten::feature_dropout encountered 27 time(s)
Unsupported operator aten::sub encountered 2 time(s)
Unsupported operator aten::add encountered 27 time(s)
Unsupported operator aten::max_unpool2d encountered 2 time(s)
Total FLOPs: 3.96 GFLOPs
Estimated time: 0.006169 seconds
/content/drive/MyDrive/cityscapes/leftImg8bit/val /content/drive/MyDrive/cityscapes/gtFine/val
100% 500/500 [01:59<00:00,  4.17it/s]
Avg inference time per image: 0.0207 seconds
Total inference time (model only): 10.33 seconds for 500 images
Per-C